# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [93]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [94]:
stock_df = pd.DataFrame()

files_m = os.listdir('../data/test_label/')

for min_df in tqdm(files_m[:10]):
    _df = pd.read_csv(f'../data/test_label/{min_df}', index_col='Unnamed: 0')
    stock_df = pd.concat([stock_df, _df])

100%|██████████| 10/10 [00:04<00:00,  2.10it/s]


In [95]:
# 첫 5줄을 봅니다.
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률,label
20221103,1530,26450,26450,26450,26450,0,9183,242880000,56069,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26450
20221103,1520,26700,26700,26600,26700,0,603,16090000,56069,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1519,26600,26700,26550,26700,0,803,21380000,55878,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1518,26600,26600,26550,26600,0,166,4410000,55848,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1517,26550,26600,26550,26600,0,1048,27870000,55814,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201028,905,16012,16089,16012,16051,0,10926,175340000,36738,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,16206
20201028,904,15973,16051,15973,16051,0,7091,113530000,32757,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,16206
20201028,903,15818,15973,15818,15973,0,20832,330780000,32415,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,16206
20201028,902,15857,15857,15779,15818,0,17369,274670000,30883,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,16206


In [96]:
def data_type_trans(stock_df):

    stock_df = stock_df.astype({'시간':'uint16'})
    stock_df = stock_df.astype({'시가':'uint32'})
    stock_df = stock_df.astype({'고가':'uint32'})
    stock_df = stock_df.astype({'저가':'uint32'})
    stock_df = stock_df.astype({'종가':'uint32'})
    stock_df = stock_df.astype({'전일대비':'int32'})
    stock_df = stock_df.astype({'거래량':'uint32'})
    stock_df = stock_df.astype({'거래대금':'uint64'})
    stock_df = stock_df.astype({'누적체결매도수량':'uint64'})
    stock_df = stock_df.astype({'상장주식수':'uint64'})
    stock_df = stock_df.astype({'시가총액':'uint64'})
    stock_df = stock_df.astype({'외국인주문한도수량':'uint64'})
    stock_df = stock_df.astype({'외국인주문가능수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유비율':'float32'})
    stock_df = stock_df.astype({'수정주가일자':'uint32'})
    stock_df = stock_df.astype({'수정주가비율':'float32'})
    stock_df = stock_df.astype({'기관순매수량':'int64'})
    stock_df = stock_df.astype({'기관누적순매수량':'int64'})
    stock_df = stock_df.drop(['등락주선', '등락비율', '예탁금', '주식회전율', '거래성립률'], axis=1)
    stock_df = stock_df.astype({'label':'uint32'})

    stock_df.info()

    return stock_df

In [97]:
stock_df = data_type_trans(stock_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897741 entries, 20221103 to 20201028
Data columns (total 20 columns):
 #   Column     Dtype  
---  ------     -----  
 0   시간         uint16 
 1   시가         uint32 
 2   고가         uint32 
 3   저가         uint32 
 4   종가         uint32 
 5   전일대비       int32  
 6   거래량        uint32 
 7   거래대금       uint64 
 8   누적체결매도수량   uint64 
 9   상장주식수      uint64 
 10  시가총액       uint64 
 11  외국인주문한도수량  uint64 
 12  외국인주문가능수량  uint64 
 13  외국인현보유수량   uint64 
 14  외국인현보유비율   float32
 15  수정주가일자     uint32 
 16  수정주가비율     float32
 17  기관순매수량     int64  
 18  기관누적순매수량   int64  
 19  label      uint32 
dtypes: float32(2), int32(1), int64(2), uint16(1), uint32(7), uint64(7)
memory usage: 220.8 MB


In [98]:
trans = stock_df.loc[:,['고가','label']]
trans = trans.rename(columns={'label':'pct_label'}).T

In [99]:
get_trans = trans.pct_change().T.iloc[:,-1]
get_trans

20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.003759
20221103    0.003759
              ...   
20201028    0.007272
20201028    0.009657
20201028    0.014587
20201028    0.022009
20201028    0.012116
Name: pct_label, Length: 1897741, dtype: float64

In [103]:
stock_df = pd.concat([stock_df, get_trans], axis=1)
stock_df = stock_df.drop(['label'], axis=1)
stock_df


,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,26450,26450,26450,26450,0,9183,242880000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1520,26700,26700,26600,26700,0,603,16090000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1519,26600,26700,26550,26700,0,803,21380000,55878,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1518,26600,26600,26550,26600,0,166,4410000,55848,0,0,0,0,0,0.0,0,0.0,0,0,0.003759
20221103,1517,26550,26600,26550,26600,0,1048,27870000,55814,0,0,0,0,0,0.0,0,0.0,0,0,0.003759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201028,905,16012,16089,16012,16051,0,10926,175340000,36738,0,0,0,0,0,0.0,0,0.0,0,0,0.007272
20201028,904,15973,16051,15973,16051,0,7091,113530000,32757,0,0,0,0,0,0.0,0,0.0,0,0,0.009657
20201028,903,15818,15973,15818,15973,0,20832,330780000,32415,0,0,0,0,0,0.0,0,0.0,0,0,0.014587
20201028,902,15857,15857,15779,15818,0,17369,274670000,30883,0,0,0,0,0,0.0,0,0.0,0,0,0.022009


In [ ]:
get_trans = trans.pct_change().T.iloc[:,-1]
get_trans


In [ ]:
get_trans.rename({'label':'tct_label'})
get_trans

In [35]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]

In [36]:
X_stock_sc = min_max_scaler.fit_transform(X_stock_df)
print(X_stock_sc)

[[8.62825789e-01 2.39009261e-02 2.37925513e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.49108368e-01 2.44431431e-02 2.43342267e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.47736626e-01 2.42262563e-02 2.43342267e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.74348422e-03 8.41520810e-04 1.09201765e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.37174211e-03 9.26106665e-04 8.40680258e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.17769536e-03 1.17651902e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [37]:
X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
X_stock_trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897741 entries, 0 to 1897740
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   시간         float64
 1   시가         float64
 2   고가         float64
 3   저가         float64
 4   종가         float64
 5   전일대비       float64
 6   거래량        float64
 7   거래대금       float64
 8   누적체결매도수량   float64
 9   상장주식수      float64
 10  시가총액       float64
 11  외국인주문한도수량  float64
 12  외국인주문가능수량  float64
 13  외국인현보유수량   float64
 14  외국인현보유비율   float64
 15  수정주가일자     float64
 16  수정주가비율     float64
 17  기관순매수량     float64
 18  기관누적순매수량   float64
dtypes: float64(19)
memory usage: 275.1 MB


## 3. 원-핫 인코딩

In [5]:
# 속성을 X, 클래스를 y로 저장합니다.
X = stock_df.iloc[:,0:-1]
y = stock_df.iloc[:,-1]

# X와 y의 첫 5줄을 출력해 보겠습니다. 
print(X[0:5])
print(y[0:5])

            시간     시가     고가     저가     종가  전일대비   거래량       거래대금  누적체결매도수량  \
20221103  1530  26450  26450  26450  26450     0  9183  242880000     56069   
20221103  1520  26700  26700  26600  26700     0   603   16090000     56069   
20221103  1519  26600  26700  26550  26700     0   803   21380000     55878   
20221103  1518  26600  26600  26550  26600     0   166    4410000     55848   
20221103  1517  26550  26600  26550  26600     0  1048   27870000     55814   

          상장주식수  ...  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  기관누적순매수량  등락주선  등락비율  \
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   

          예탁금  주식회전율  거래성립률  
20221103    0 

In [6]:
# 원-핫 인코딩 처리를 합니다.
y = pd.get_dummies(y)

# 원-핫 인코딩 결과를 확인합니다.
print(y[0:5])

          15469   15585   15624   15702   15740   15779   15818   15857   \
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   

          15896   15934   ...  470000  470500  471000  471500  472500  473000  \
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   

          474000  474500  475500  477000  
20221103     

In [7]:
y.value_counts()

KeyboardInterrupt: 

## 4. 소프트맥스

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델 설정
model = Sequential()
model.add(Dense(12,  input_dim=4, activation='relu'))
model.add(Dense(8,  activation='relu')) # 주식에서는 relu를 쓰면 안된다.(변동률이 마이너스가 가능하기 때문에)
# 주식에서는 relu 대신에 linear를 쓰는 경우가 많다.
model.add(Dense(8,  activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(3, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X, y, epochs=30, batch_size=5)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 12)                60        
                                                                 
 dense_14 (Dense)            (None, 8)                 104       
                                                                 
 dense_15 (Dense)            (None, 8)                 72        
                                                                 
 dense_16 (Dense)            (None, 8)                 72        
                                                                 
 dense_17 (Dense)            (None, 8)                 72        
                                                                 
 dense_18 (Dense)            (None, 3)                 27        
                                                                 
Total params: 407
Trainable params: 407
Non-trainable 

## 5. 아이리스 품족 예측 실행

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 깃허브에 준비된 데이터를 가져옵니다. 앞에서 이미 가져왔으므로 주석 처리합니다. 5번 예제만 별도 실행 시 주석을 해제한 후 실행해주세요.
# !git clone https://github.com/taehojo/data.git

# 아이리스 데이터를 불러옵니다.
df = pd.read_csv('./data/iris3.csv')

# 속성을 X, 클래스를 y로 저장합니다.
X = df.iloc[:,0:4]
y = df.iloc[:,4]

# 원-핫 인코딩 처리를 합니다.
y = pd.get_dummies(y)

# 모델 설정
model = Sequential()
model.add(Dense(12,  input_dim=4, activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X, y, epochs=30, batch_size=5)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 12)                60        
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
30/30 [==============================] - 0s 2ms/step - loss: 1.4888 - accuracy: 0.3333
Epoch 2/30
30/30 [==============================] - 0s 2ms/step - loss: 1.0854 - accuracy: 0.4067
Epoch 3/30
30/30 [==============================] - 0s 2ms/step - loss: 0.9764 - accuracy: 0.6000
Epoch 4/30
30/30 [==============================] - 0s 2ms/step - loss: 0.9382 - accuracy: 0